In [73]:
import csv
import pandas as pd
import operator
#from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor # you have to install it first!

In [2]:
sample = pd.read_csv(r'all\sample_submission.csv', sep = ',')
sample.head(5)

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


# Test file

In [3]:
test = pd.read_csv(r'all\test.csv', sep = ',')
test.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
5,1466,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
6,1467,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal
7,1468,60,RL,63.0,8402,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
8,1469,20,RL,85.0,10176,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal
9,1470,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,4,2010,WD,Normal


In [4]:
len(test)

1459

In [6]:
test.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,...,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,...,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,...,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,...,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,...,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,...,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,...,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [7]:
# set of columns with at least one NaN value
nan_dict = {}
for col in test:
    nan_count = test[test[col].isna()]
    if len(nan_count)>0:
        nan_dict[col] = len(nan_count)
        
sorted_nan = sorted(nan_dict.items(), key = operator.itemgetter(1))

In [8]:
sorted_nan.reverse()
sorted_nan

[('PoolQC', 1456),
 ('MiscFeature', 1408),
 ('Alley', 1352),
 ('Fence', 1169),
 ('FireplaceQu', 730),
 ('LotFrontage', 227),
 ('GarageCond', 78),
 ('GarageQual', 78),
 ('GarageFinish', 78),
 ('GarageYrBlt', 78),
 ('GarageType', 76),
 ('BsmtCond', 45),
 ('BsmtExposure', 44),
 ('BsmtQual', 44),
 ('BsmtFinType2', 42),
 ('BsmtFinType1', 42),
 ('MasVnrType', 16),
 ('MasVnrArea', 15),
 ('MSZoning', 4),
 ('Functional', 2),
 ('BsmtHalfBath', 2),
 ('BsmtFullBath', 2),
 ('Utilities', 2),
 ('SaleType', 1),
 ('GarageArea', 1),
 ('GarageCars', 1),
 ('KitchenQual', 1),
 ('TotalBsmtSF', 1),
 ('BsmtUnfSF', 1),
 ('BsmtFinSF2', 1),
 ('BsmtFinSF1', 1),
 ('Exterior2nd', 1),
 ('Exterior1st', 1)]

In [26]:
# creating a set of columns names (with NaN values in test)
test_li = set()
for i in range(len(sorted_nan)):
    test_li.add(sorted_nan[i][0])
test_li

{'Alley',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFinType1',
 'BsmtFinType2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtQual',
 'BsmtUnfSF',
 'Exterior1st',
 'Exterior2nd',
 'Fence',
 'FireplaceQu',
 'Functional',
 'GarageArea',
 'GarageCars',
 'GarageCond',
 'GarageFinish',
 'GarageQual',
 'GarageType',
 'GarageYrBlt',
 'KitchenQual',
 'LotFrontage',
 'MSZoning',
 'MasVnrArea',
 'MasVnrType',
 'MiscFeature',
 'PoolQC',
 'SaleType',
 'TotalBsmtSF',
 'Utilities'}

**Alley** has 107 null values and the remaining are NaN.  ---> drop ?

**MiscFeature** has 3 second garage, 2 other, 46 shed and the remaining 1408 are missing. We can drop it since we don't need these informations. 

**MiscVal** has 51 different values and the remaining 1408 are missing (*except* miscval = 17000, miscfeature = nan).

**PoolArea** has 1453 zeroes and the remaining 6 are different values. 

**PoolQC** has 2 excellent and 1 good values, the remaining 3 are NaN. The information is not reliable.  ---> drop

In [10]:
col = test['PoolArea']
grouped = col.groupby(col).count()
grouped

PoolArea
0      1453
144       1
228       1
368       1
444       1
561       1
800       1
Name: PoolArea, dtype: int64

In [11]:
# returns the dataframe rows which satisfy the condition 
test.loc[test['MiscVal'] > 0]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
6,1467,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal
56,1517,85,RL,75.0,11380,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Shed,1500,1,2010,WD,Normal
58,1519,20,RL,70.0,8197,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,300,2,2010,WD,Normal
108,1569,50,RL,124.0,18600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,Shed,450,6,2010,WD,Normal
133,1594,90,RH,60.0,7200,Pave,Pave,Reg,Lvl,AllPub,...,0,0,NaN,NaN,Shed,500,2,2010,WD,Normal
145,1606,80,RL,74.0,9620,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,80,5,2010,WD,Normal
159,1620,90,RL,70.0,7700,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,Shed,600,2,2010,WD,Normal
168,1629,80,RL,60.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,Shed,490,6,2009,WD,Normal
199,1660,20,RL,NaN,10710,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,Shed,1200,7,2009,WD,Normal


# Train

In [83]:
train = pd.read_csv(r'all\train.csv', sep = ',')
train.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [19]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [20]:
# set of columns with at least one NaN value
nan_dict_train = {}
for col in train:
    nan_count_train = train[train[col].isna()]
    if len(nan_count_train)>0:
        nan_dict_train[col] = len(nan_count_train)
        
sorted_nan_train = sorted(nan_dict_train.items(), key = operator.itemgetter(1))

In [21]:
sorted_nan_train.reverse()
sorted_nan_train

[('PoolQC', 1453),
 ('MiscFeature', 1406),
 ('Alley', 1369),
 ('Fence', 1179),
 ('FireplaceQu', 690),
 ('LotFrontage', 259),
 ('GarageCond', 81),
 ('GarageQual', 81),
 ('GarageFinish', 81),
 ('GarageYrBlt', 81),
 ('GarageType', 81),
 ('BsmtFinType2', 38),
 ('BsmtExposure', 38),
 ('BsmtFinType1', 37),
 ('BsmtCond', 37),
 ('BsmtQual', 37),
 ('MasVnrArea', 8),
 ('MasVnrType', 8),
 ('Electrical', 1)]

In [24]:
# creating a set of columns names (with NaN values in train)
train_li = set()
for i in range(len(sorted_nan_train)):
    train_li.add(sorted_nan[i][0])
train_li

{'Alley',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'BsmtQual',
 'Fence',
 'FireplaceQu',
 'GarageCond',
 'GarageFinish',
 'GarageQual',
 'GarageType',
 'GarageYrBlt',
 'LotFrontage',
 'MSZoning',
 'MasVnrArea',
 'MasVnrType',
 'MiscFeature',
 'PoolQC'}

In [30]:
# all the categories (19) in train_li are also in test_li (33)
print(train_li - test_li)

set()


# Modelling

In [32]:
reduced_test = test.drop(test_li, axis = 1)
reduced_test.head()

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
0,1461,20,11622,Pave,Reg,Lvl,Inside,Gtl,NAmes,Feedr,...,140,0,0,0,120,0,0,6,2010,Normal
1,1462,20,14267,Pave,IR1,Lvl,Corner,Gtl,NAmes,Norm,...,393,36,0,0,0,0,12500,6,2010,Normal
2,1463,60,13830,Pave,IR1,Lvl,Inside,Gtl,Gilbert,Norm,...,212,34,0,0,0,0,0,3,2010,Normal
3,1464,60,9978,Pave,IR1,Lvl,Inside,Gtl,Gilbert,Norm,...,360,36,0,0,0,0,0,6,2010,Normal
4,1465,120,5005,Pave,IR1,HLS,Inside,Gtl,StoneBr,Norm,...,0,82,0,0,144,0,0,1,2010,Normal


In [84]:
reduced_train = train.drop(test_li, axis = 1)
reduced_train = reduced_train.drop('SalePrice', axis = 1)
reduced_train.head()

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
0,1,60,8450,Pave,Reg,Lvl,Inside,Gtl,CollgCr,Norm,...,0,61,0,0,0,0,0,2,2008,Normal
1,2,20,9600,Pave,Reg,Lvl,FR2,Gtl,Veenker,Feedr,...,298,0,0,0,0,0,0,5,2007,Normal
2,3,60,11250,Pave,IR1,Lvl,Inside,Gtl,CollgCr,Norm,...,0,42,0,0,0,0,0,9,2008,Normal
3,4,70,9550,Pave,IR1,Lvl,Corner,Gtl,Crawfor,Norm,...,0,35,272,0,0,0,0,2,2006,Abnorml
4,5,60,14260,Pave,IR1,Lvl,FR2,Gtl,NoRidge,Norm,...,192,84,0,0,0,0,0,12,2008,Normal


In [66]:
# converting the categorical data of test
test_predictors = pd.get_dummies(reduced_test)
test_predictors.head()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,...,Electrical_SBrkr,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,11622,5,6,1961,1961,896,0,0,...,1,0,0,1,0,0,0,0,1,0
1,1462,20,14267,6,6,1958,1958,1329,0,0,...,1,0,0,1,0,0,0,0,1,0
2,1463,60,13830,5,5,1997,1998,928,701,0,...,1,0,0,1,0,0,0,0,1,0
3,1464,60,9978,6,6,1998,1998,926,678,0,...,1,0,0,1,0,0,0,0,1,0
4,1465,120,5005,8,5,1992,1992,1280,0,0,...,1,0,0,1,0,0,0,0,1,0


In [85]:
# converting the categorical data of train
train_predictors = pd.get_dummies(reduced_train)
train_predictors.head()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,...,Electrical_SBrkr,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,8450,7,5,2003,2003,856,854,0,...,1,0,0,1,0,0,0,0,1,0
1,2,20,9600,6,8,1976,1976,1262,0,0,...,1,0,0,1,0,0,0,0,1,0
2,3,60,11250,7,5,2001,2002,920,866,0,...,1,0,0,1,0,0,0,0,1,0
3,4,70,9550,7,5,1915,1970,961,756,0,...,1,0,0,1,1,0,0,0,0,0
4,5,60,14260,8,5,2000,2000,1145,1053,0,...,1,0,0,1,0,0,0,0,1,0


In [86]:
# to ensure the test data is encoded in the same manner
final_train_X, final_test_X = train_predictors.align(test_predictors,
                                                 join = 'left', axis = 1)

In [90]:
# in order to isolate the price
final_train_y = train.SalePrice

1460

In [92]:
len(final_train_X)

1460

In [96]:
my_model = XGBRegressor(n_estimators = 1000, learning_rate=0.05)
my_model.fit(final_train_X, final_train_y)#, early_stopping_rounds = 5)
             #, eval_set = [(test_X, test_y)], verbose = False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [99]:
a = my_model.predict(final_test_X)

#for price in a:
#    print(price)
a

array([120927.02 , 152043.8  , 180440.11 , ..., 130881.016, 127618.34 ,
       225014.9  ], dtype=float32)

In [100]:
len(a)

1459

In [120]:
pred_sale_price = pd.DataFrame(a, columns = ['SalePrice'], index=None)
pred_sale_price['Id'] = test['Id']
pred_sale_price.head()

,SalePrice,Id
0,120927.023438,1461
1,152043.796875,1462
2,180440.109375,1463
3,196468.218750,1464
4,195479.359375,1465


In [115]:
cols = pred_sale_price.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols

['Id', 'SalePrice']

In [121]:
pred_sale_price = pred_sale_price[cols]
pred_sale_price

,Id,SalePrice
0,1461,120927.023438
1,1462,152043.796875
2,1463,180440.109375
3,1464,196468.218750
4,1465,195479.359375
5,1466,184104.578125
6,1467,162368.062500
7,1468,174345.171875
8,1469,173780.234375
9,1470,118976.093750


In [123]:
pred_sale_price.to_csv('predictions.csv', index=False)